In [3]:
import cv2
import numpy as np

def detect_icons_yolo(image_path, yolo_net, yolo_layer_names, threshold=0.5):
    img = cv2.imread(image_path)
    height, width, channels = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    # Pass the image blob through the network
    yolo_net.setInput(blob)
    outs = yolo_net.forward(yolo_layer_names)

    class_ids = []
    confidences = []
    boxes = []

    # Process detections
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Define rectangle coordinates
                x = center_x - w // 2
                y = center_y - h // 2
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maxima suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, threshold, 0.4)

    # Draw results on image
    for i in indices.flatten():
        box = boxes[i]
        x, y, w, h = box
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    return img

In [4]:
output_folder = 'C:/Users/pis05408.PINNACLE/Desktop/Suraj/ObjectDetectionFlorPlan/Test'
os.makedirs(output_folder, exist_ok=True)
print(f'output_folder: {output_folder}')

output_folder: C:/Users/pis05408.PINNACLE/Desktop/Suraj/ObjectDetectionFlorPlan/Test


In [5]:
# Load your target image and templates
image_path = 'data/sample.jpg'
# template_paths = ['data/test.jpg', 'data/test1.jpg', 'data/test1.jpg', 'data/test2.jpg', 'data/test3.jpg', 'data/test4.jpg', 'data/test5.jpg', 'data/test6.jpg']
template_paths = ['data/test.jpg']

# Read the images
image = cv2.imread(image_path)
templates = [cv2.imread(template_path) for template_path in template_paths]

In [7]:
# Load YOLO model (need pre-trained weights and config files)
net = cv2.dnn.readNet("yolov4.weights", "yolov4.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Use the YOLO model to detect icons
detect_icons_yolo("floor_plan.png", net, output_layers)

error: OpenCV(4.10.0) C:\b\abs_daut97tdpo\croot\opencv-suite_1722029138522\work\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov4.cfg in function 'cv::dnn::dnn4_v20240521::readNetFromDarknet'


###

In [8]:
import cv2

# Load images
floor_plan = cv2.imread('data/sample.jpg', cv2.IMREAD_GRAYSCALE)
icon = cv2.imread('data/test.jpg', cv2.IMREAD_GRAYSCALE)

# Initialize ORB detector
orb = cv2.ORB_create()

# Find keypoints and descriptors
keypoints1, descriptors1 = orb.detectAndCompute(floor_plan, None)
keypoints2, descriptors2 = orb.detectAndCompute(icon, None)

# Create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match descriptors
matches = bf.match(descriptors1, descriptors2)

# Sort them in ascending order of distance
matches = sorted(matches, key=lambda x: x.distance)

# Draw matches
img_matches = cv2.drawMatches(floor_plan, keypoints1, icon, keypoints2, matches[:10], None)
# cv2.imshow('Matches', img_matches)
# cv2.waitKey(0)

# Optionally, save the detected image
cv2.imwrite('output/orb.jpg', img_matches)

error: OpenCV(4.10.0) C:\b\abs_daut97tdpo\croot\opencv-suite_1722029138522\work\modules\core\src\batch_distance.cpp:275: error: (-215:Assertion failed) type == src2.type() && src1.cols == src2.cols && (type == CV_32F || type == CV_8U) in function 'cv::batchDistance'


In [ ]:
# def rotate_image(image, angle):
#     (h, w) = image.shape[:2]
#     center = (w / 2, h / 2)
#     M = cv2.getRotationMatrix2D(center, angle, 1.0)
#     cos = np.abs(M[0, 0])
#     sin = np.abs(M[0, 1])
#     new_w = int((h * sin) + (w * cos))
#     new_h = int((h * cos) + (w * sin))
#     M[0, 2] += (new_w / 2) - center[0]
#     M[1, 2] += (new_h / 2) - center[1]
#     rotated = cv2.warpAffine(image, M, (new_w, new_h))
#     return rotated

# def multi_scale_and_angle_template_matching(image, templates, threshold=0.7):
#     bounding_boxes = []

#     for template in templates:
#         h, w = template.shape[:2]
        
#         # Loop over scales
#         for scale in np.linspace(0.5, 1.5, num=10):  # Adjust scale range as needed
#             resized_template = cv2.resize(template, (int(w * scale), int(h * scale)))
#             res_h, res_w = resized_template.shape[:2]

#             # Skip if resized template is larger than the image
#             if res_h > image.shape[0] or res_w > image.shape[1]:
#                 continue

#             # Loop over angles
#             for angle in [90, 180, 270]:  # Adjust angle step as needed
#                 M = cv2.getRotationMatrix2D((res_w / 2, res_h / 2), angle, 1.0)
#                 rotated_template = cv2.warpAffine(resized_template, M, (res_w, res_h))

#                 # Template matching
#                 result = cv2.matchTemplate(image, rotated_template, cv2.TM_CCOEFF_NORMED)
#                 loc = np.where(result >= threshold)

#                 # Record all locations where matches are found
#                 for pt in zip(*loc[::-1]):  # Switch x and y coordinates
#                     bounding_boxes.append((pt[0], pt[1], res_w, res_h))  # (x, y, width, height)

#     return bounding_boxes

# def non_max_suppression(boxes, overlapThresh=0.7):
#     if len(boxes) == 0:
#         return []

#     boxes = np.array(boxes)
#     pick = []

#     x1 = boxes[:, 0]
#     y1 = boxes[:, 1]
#     x2 = boxes[:, 0] + boxes[:, 2]
#     y2 = boxes[:, 1] + boxes[:, 3]

#     area = (x2 - x1) * (y2 - y1)
#     idxs = np.argsort(y2)

#     while len(idxs) > 0:
#         last = len(idxs) - 1
#         i = idxs[last]
#         pick.append(i)

#         xx1 = np.maximum(x1[i], x1[idxs[:last]])
#         yy1 = np.maximum(y1[i], y1[idxs[:last]])
#         xx2 = np.minimum(x2[i], x2[idxs[:last]])
#         yy2 = np.minimum(y2[i], y2[idxs[:last]])

#         w = np.maximum(0, xx2 - xx1)
#         h = np.maximum(0, yy2 - yy1)
#         overlap = (w * h) / area[idxs[:last]]

#         idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

#     return boxes[pick].astype("int")

In [ ]:
# # Perform multi-scale and multi-angle template matching
# bounding_boxes = multi_scale_and_angle_template_matching(image, templates)

# # Apply non-maxima suppression to reduce overlapping boxes
# filtered_boxes = non_max_suppression(bounding_boxes)

# # Draw bounding boxes on the original image for visualization
# for (x, y, w, h) in filtered_boxes:
#     cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)


# # Create a filename based on scale and threshold
# output_folder = 'output'  # Specify your output folder
# os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist
# filename = f"detected_scale_1.jpg"
# filepath = os.path.join(output_folder, filename)

# # Save the detected image
# cv2.imwrite(filepath, image)

# # Print the count of detected items
# print(f'File Stored at: {filepath}\nfile_name: {filename}')

# # Show the result
# cv2.imshow("Detected Icons", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()